In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session



In [ ]:
!pip install libpysal
!pip install https://github.com/pysal/mgwr/archive/master.zip
    

In [ ]:
import numpy as np
import pandas as pd
import libpysal as ps
from mgwr.gwr import GWR, MGWR
from mgwr.sel_bw import Sel_BW
from mgwr.utils import compare_surfaces, truncate_colormap
import geopandas as gp 
import matplotlib.pyplot as plt
import matplotlib as mpl

In [ ]:
UTLA_X_Y = pd.read_csv('../input/newutla-x-y/UTLA_X_Y.csv')

counties_pop = gp.read_file('../input/new-gwr-processing/counties_pop.shp')
fig, ax = plt.subplots(figsize=(10,10))
counties_pop.plot(ax=ax, **{'edgecolor':'red', 'facecolor':'white'})
counties_pop.centroid.plot(ax=ax, c='blue')

In [ ]:
#build our GWR model
g_y = UTLA_X_Y['total_case'].values.reshape((-1,1))
g_X = UTLA_X_Y[['population density','population']].values
u = UTLA_X_Y['X']
v =UTLA_X_Y['Y']
g_coords = list(zip(u,v))
g_X = (g_X - g_X.mean(axis=0)) / g_X.std(axis=0)
g_y = g_y.reshape((-1,1))
y_mean=g_y.mean(axis=0)
y_std=g_y.std(axis=0)
g_y = (g_y - g_y.mean(axis=0)) / g_y.std(axis=0)
#tune the parameters of our GWR model
#Examples of optimal bandwidth selection
#Instantiate bandwidth selection object
gwr_selector =Sel_BW(g_coords, g_y, g_X)
gwr_bw = gwr_selector.search(bw_min=2)
print(gwr_bw)
gwr_results = GWR(g_coords, g_y, g_X, gwr_bw).fit()
yhat=gwr_results.predy
ys=yhat*y_std+y_mean
UTLA_X_Y['yhat']=ys
pd.DataFrame(UTLA_X_Y).to_csv("result.csv")

In [ ]:
gwr_results.summary()